In [72]:
import pandas as pd
import numpy as np
import random
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import load_model
import keras


In [152]:
def eval_keras(df_name: str, model_name: str) -> list:
    # read data
    df = pd.read_csv(df_name)
    X = df.drop('Outcome', axis=1)
    y = df['Outcome']

    # convert data to arrays
    X = np.array(X)
    y = np.array(y)

    # split data into the appropriate groups
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    # scale X data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # load the saved model
    model = load_model(model_name)

    # read the data for the predictions to be compared to
    real_X = pd.read_csv(df_name).drop('Outcome', axis=1)
    real_y = pd.read_csv(df_name)['Outcome']

    # loop through the test data and get recommended bets
    winning_bets = []
    losing_bets = []
    for i in range(len(real_X)):
        try:
            prediction = model.predict(scaler.transform(real_X.iloc[i].values.reshape(1, -1)), verbose=0)
            actual = real_y.iloc[i]
            if prediction > .6 and actual == 1:
                winning_bets.append([real_X.iloc[i].loc['Column501'], prediction.item()])
            elif prediction > .6 and actual == 0:
                losing_bets.append([real_X.iloc[i].loc['Column501'], prediction.item()])
        except:
            pass
    return [winning_bets, losing_bets]

In [149]:
import torch
import torch.nn as nn


def eval_pytorch(df_name: str, model_name: str) -> list:
    # read data
    df = pd.read_csv(df_name)
    X = df.drop('Outcome', axis=1)
    y = df['Outcome']

    # convert data to arrays
    X = np.array(X)
    y = np.array(y)

    # split data into the appropriate groups
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    # scale X data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    class ANN_Model(nn.Module):
        def __init__(self, input_size):
            super(ANN_Model, self).__init__()
            self.fc1 = nn.Linear(input_size, 256)
            self.fc2 = nn.Linear(256, 128)
            self.fc3 = nn.Linear(128, 64)
            self.fc4 = nn.Linear(64, 32)
            self.fc5 = nn.Linear(32, 1)
            self.relu = nn.ReLU()
            self.dropout = nn.Dropout(0.1)

        def forward(self, x):
            x = self.relu(self.fc1(x))
            x = self.dropout(x)
            x = self.relu(self.fc2(x))
            x = self.dropout(x)
            x = self.relu(self.fc3(x))
            x = self.dropout(x)
            x = self.relu(self.fc4(x))
            x = self.dropout(x)
            x = self.fc5(x)
            return x

    # load the saved model
    model = ANN_Model(input_size=X_train.shape[1])
    model.load_state_dict(torch.load(model_name))
    device = torch.device("cuda")
    model.to(device)
    model.eval()

    # loop through the test data and get recommended bets
    real_X = pd.read_csv(df_name).drop('Outcome', axis=1)
    real_y = pd.read_csv(df_name)['Outcome']

    # loop through the test data and get recommended bets
    winning_bets = []
    losing_bets = []
    for i in range(len(real_X)):
        X_tensor = torch.tensor(scaler.transform(np.array(real_X.iloc[i].values.reshape(1, -1))), dtype=torch.float32).cuda().to(device)
        y_tensor = torch.tensor([real_y.iloc[i]], dtype=torch.float32).cuda().to(device)
        outputs = model(X_tensor)
        prediction = torch.sigmoid(outputs).item()
        actual = real_y.iloc[i]

        if prediction > 0.6 and actual == 1:
            winning_bets.append([real_X.iloc[i].loc['Column501'], prediction])
        elif prediction > 0.6 and actual == 0:
            losing_bets.append([real_X.iloc[i].loc['Column501'], prediction])

    return [winning_bets, losing_bets]


In [ ]:
def calculate_winnings(bet_amount, odds):
    potential_winnings = bet_amount * (odds / 100)
    return potential_winnings


def american_to_decimal(american_odds):
    if american_odds >= 100:
        decimal_odds = (american_odds / 100)
    else:
        decimal_odds = (100 / abs(american_odds))
    return round(decimal_odds, 2)


def calculate_kelly_criterion(american_odds, model_prob):
    decimal_odds = american_to_decimal(american_odds)
    bankroll_fraction = round((100 * (decimal_odds * model_prob - (1 - model_prob))) / decimal_odds, 2)
    return bankroll_fraction if bankroll_fraction > 0 else 0

In [153]:
a = eval_keras("csvs/new_ml_X.csv", "built_models/keras_ml.h5")
print(len(a[0]), len(a[1]))
print(a)
# make sure to change the column numbers for the other types of data

101 60
[[[165.0, 0.6206771731376648], [155.0, 0.6217005252838135], [-170.0, 0.6749032735824585], [-205.0, 0.6854712963104248], [-300.0, 0.7184167504310608], [-330.0, 0.7264301776885986], [-185.0, 0.68421471118927], [-164.0, 0.6791132688522339], [-220.0, 0.6146557331085205], [-240.0, 0.6185315251350403], [-180.0, 0.6918414831161499], [-180.0, 0.6918423175811768], [-315.0, 0.6359438300132751], [-350.0, 0.6492761373519897], [-165.0, 0.6969630718231201], [-124.0, 0.6864391565322876], [-210.0, 0.6436251401901245], [-255.0, 0.6489968299865723], [-145.0, 0.8539546728134155], [-170.0, 0.8654806613922119], [-130.0, 0.6290745735168457], [-158.0, 0.6338067054748535], [-105.0, 0.6667555570602417], [-105.0, 0.6667750477790833], [-105.0, 0.6141589283943176], [-185.0, 0.6853040456771851], [-156.0, 0.6741654872894287], [-115.0, 0.6128230690956116], [-150.0, 0.6278670430183411], [-145.0, 0.6869640350341797], [-170.0, 0.6957559585571289], [-300.0, 0.6374648809432983], [-270.0, 0.6302694082260132], [-200

In [154]:
b = eval_pytorch("csvs/new_ml_X.csv", "built_models/torch_ml.pth")
print(len(b[0]), len(b[1]))
print(b)

289 206
[[[-170.0, 0.666241466999054], [-205.0, 0.7071399092674255], [165.0, 0.9795644879341125], [155.0, 0.9800559878349304], [-170.0, 0.6749266982078552], [-205.0, 0.7073240876197815], [-125.0, 0.7525908350944519], [-170.0, 0.763505756855011], [-185.0, 0.7449747323989868], [-164.0, 0.7047755122184753], [145.0, 0.7475247979164124], [125.0, 0.7653805613517761], [100.0, 0.8554372787475586], [122.0, 0.839231550693512], [-140.0, 0.9104218482971191], [-130.0, 0.9102153182029724], [-125.0, 0.6795079112052917], [-115.0, 0.6775590777397156], [-180.0, 0.8170866370201111], [-180.0, 0.8171574473381042], [-315.0, 0.922487199306488], [-350.0, 0.9323952198028564], [-170.0, 0.6950920224189758], [-180.0, 0.700103759765625], [-200.0, 0.6822628974914551], [-185.0, 0.6742278933525085], [115.0, 0.902857780456543], [132.0, 0.9015715718269348], [-180.0, 0.9734155535697937], [-170.0, 0.9705110192298889], [-120.0, 0.8516102433204651], [-110.0, 0.8446900248527527], [-110.0, 0.8923659920692444], [-102.0, 0.890